## Vegetation Prediction using Sentinel-2

In this computing lab, you will create a simple machine learning model to predict vegetation percentage from Sentinel-2 imagery. The Copernicus Sentinel-2 mission is organized as a constellation of identical satellites in the same orbit: Sentinel-2A launched in 2015, Sentinel-2B launched in 2017, and Sentinel-2C launched in September of this year. Each satellite carries a multispectral imager with 13 spectral bands to measure our land and vegetation.

<img src=sentinel-2.jpg alt="Sentinel 2" width=500>

### 1. Learning Objectives
The goals of this computing lab are to:

1. Familiarize yourself with the structure of Sentinel-2 data
2. Visualize sample Sentinel-2 imagery and corresponding vegetation labels
3. Create a supervised machine learning pipeline using Pytorch Lightning and develop
    - dataloaders
    - data augmentations & transformations
    - (one or more) machine learning models
    - validation metrics